In [1]:
import os
import torch

# os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

import datasets

from peft import get_peft_model
from peft import LoraConfig
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq, DataCollatorWithPadding
from transformers import Trainer
from transformers import TrainingArguments

from transformers.utils.logging import DEBUG
from transformers.utils.logging import set_verbosity

# set_verbosity(DEBUG)

/home/pai/envs/test_qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


you got mock peft!
you got mock transformers!


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f428d954610>>
Traceback (most recent call last):
  File "/home/pai/envs/test_qwen/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
# 加载Dataset。原文经过DataFrame中转，比较麻烦
ds = datasets.load_dataset("json", data_files="fake_data_3_v2.json", split="train")
ds[0]

KeyboardInterrupt: 

In [ ]:
CKPT_PATH = "Qwen-1_8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(CKPT_PATH, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eod_id
tokenizer

In [ ]:
def process_func(example):
    MAX_LENGTH = 384

    instruction = tokenizer(example["instruction"] + example["input"])
    response = tokenizer(example["output"])
    
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eod_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eod_id]

    # 截断
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
# 类似DataFrame的用法，并行处理数据
train_dataset = ds.map(process_func, remove_columns=ds.column_names)  # 去掉原始的列
train_dataset

In [ ]:
train_dataset['input_ids']

In [ ]:
train_dataset['attention_mask']

In [ ]:
train_dataset['labels']

In [ ]:
tokenizer.decode(train_dataset[0]['input_ids'])

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, train_dataset[0]["labels"])))

In [ ]:
model = AutoModelForCausalLM.from_pretrained(CKPT_PATH, trust_remote_code=True,
                                             # torch_dtype=torch.float16
                                            )
model

In [ ]:
model.dtype

In [ ]:
config = LoraConfig(
    # task_type=TaskType.CAUSAL_LM, # 可以不写
    target_modules=["c_attn", "c_proj", "w1", "w2"],  # 必须指定，Qwen官方的微调脚本也是写这几个
    # 原文说这三个参数比较通用，先留着
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
config

In [ ]:
model = get_peft_model(model, config)
# model

In [ ]:
model.print_trainable_parameters()

In [ ]:
# 定义训练配置
args = TrainingArguments(
    output_dir=f"./output/{CKPT_PATH}_cpu_v1",
    per_device_train_batch_size=3,
    gradient_accumulation_steps=1,
    logging_steps=1,
    num_train_epochs=1,
    # gradient_checkpointing=True,
    # save_steps=2,
    learning_rate=0.0001,
    # include_tokens_per_second=True,
    # include_num_input_tokens_seen=True
    # dataloader_drop_last=True
)

In [ ]:
# 定义trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    # 用pad填充batch的三个字段
    data_collator=DataCollatorForSeq2Seq(tokenizer),  # 原文设置padding=True，是默认值，省略掉
    # data_collator=DataCollatorWithPadding(tokenizer)
    # tokenizer=tokenizer
)

In [ ]:
output = trainer.train()
output